In [1]:
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow as tf
from tensorflow.keras.layers import *
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler, MaxAbsScaler
from sklearn.model_selection import train_test_split
from ta import add_all_ta_features
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.pyplot import step, show
from sklearn.impute import SimpleImputer
import talib
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

## Load Data

In [2]:
df = pd.read_csv('C:/Users/ali.khankan/Downloads/notebooks/LSTM/data_50K.csv',index_col=0,parse_dates=True)
#drop NaN
df.dropna(inplace=True)
#add candle patterns
candle_names = talib.get_function_groups()['Pattern Recognition']
# extract OHLC 
op = df['o']
hi = df['h']
lo = df['l']
cl = df['c']
# create columns for each pattern
for candle in candle_names:
    # below is same as;
    # df["CDL3LINESTRIKE"] = talib.CDL3LINESTRIKE(op, hi, lo, cl)
    df[candle] = getattr(talib, candle)(op, hi, lo, cl)
del op, hi, lo, cl
#add dummies
df = pd.get_dummies(data=df, columns=['minutes','hours', 'days'] + candle_names, drop_first=True)
#add TA
np.seterr(divide='ignore', invalid='ignore')
df = add_all_ta_features(df, open="o", high="h", low="l", close="c", volume="v",fillna=False)
#Deal with NaN
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
imputer = imputer.fit_transform(df)
df = pd.DataFrame(imputer, columns=df.columns.values, index=df.index )
df

C:\ProgramData\Anaconda3\lib\site-packages\ta\wrapper.py:285: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"{colprefix}trend_mass_index"] = MassIndex(
C:\ProgramData\Anaconda3\lib\site-packages\ta\wrapper.py:290: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"{colprefix}trend_dpo"] = DPOIndicator(
C:\ProgramData\Anaconda3\lib\site-packages\ta\wrapper.py:308: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joinin

,c,h,l,o,v,diff,%change,gap,returns,Cdirection,Fdirection,minutes_1,minutes_2,minutes_3,minutes_4,minutes_5,minutes_6,minutes_7,minutes_8,minutes_9,minutes_10,minutes_11,minutes_12,minutes_13,minutes_14,minutes_15,minutes_16,minutes_17,minutes_18,minutes_19,minutes_20,minutes_21,minutes_22,minutes_23,minutes_24,minutes_25,minutes_26,minutes_27,minutes_28,minutes_29,minutes_30,minutes_31,minutes_32,minutes_33,minutes_34,minutes_35,minutes_36,minutes_37,minutes_38,minutes_39,minutes_40,minutes_41,minutes_42,minutes_43,minutes_44,minutes_45,minutes_46,minutes_47,minutes_48,minutes_49,minutes_50,minutes_51,minutes_52,minutes_53,minutes_54,minutes_55,minutes_56,minutes_57,minutes_58,minutes_59,hours_1,hours_2,hours_3,hours_4,hours_5,hours_6,hours_7,hours_8,hours_9,hours_10,hours_11,hours_12,hours_13,hours_14,hours_15,hours_16,hours_17,hours_18,hours_19,hours_20,hours_21,hours_22,hours_23,days_2,days_3,days_4,days_5,days_6,days_7,days_8,days_9,days_10,days_11,days_12,days_13,days_14,days_15,days_16,days_17,days_18,days_19,days_20,days_21,days_22,days_23,days_24,days_25,days_26,days_27,days_28,days_29,days_30,CDL3BLACKCROWS_0,CDL3INSIDE_0,CDL3INSIDE_100,CDL3LINESTRIKE_0,CDL3LINESTRIKE_100,CDL3OUTSIDE_0,CDL3OUTSIDE_100,CDL3WHITESOLDIERS_100,CDLADVANCEBLOCK_0,CDLBELTHOLD_0,CDLBELTHOLD_100,CDLBREAKAWAY_100,CDLCLOSINGMARUBOZU_0,CDLCLOSINGMARUBOZU_100,CDLCONCEALBABYSWALL_100,CDLDARKCLOUDCOVER_0,CDLDOJI_100,CDLDOJISTAR_0,CDLDOJISTAR_100,CDLDRAGONFLYDOJI_100,CDLENGULFING_0,CDLENGULFING_100,CDLEVENINGDOJISTAR_0,CDLEVENINGSTAR_0,CDLGAPSIDESIDEWHITE_100,CDLGRAVESTONEDOJI_100,CDLHAMMER_100,CDLHANGINGMAN_0,CDLHARAMI_0,CDLHARAMI_100,CDLHARAMICROSS_0,CDLHARAMICROSS_100,CDLHIGHWAVE_0,CDLHIGHWAVE_100,CDLHIKKAKE_-100,CDLHIKKAKE_0,CDLHIKKAKE_100,CDLHIKKAKE_200,CDLHIKKAKEMOD_-100,CDLHIKKAKEMOD_0,CDLHIKKAKEMOD_100,CDLHIKKAKEMOD_200,CDLHOMINGPIGEON_100,CDLIDENTICAL3CROWS_0,CDLINNECK_0,CDLINVERTEDHAMMER_100,CDLLADDERBOTTOM_100,CDLLONGLEGGEDDOJI_100,CDLLONGLINE_0,CDLLONGLINE_100,CDLMARUBOZU_0,CDLMARUBOZU_100,CDLMATCHINGLOW_100,CDLMORNINGDOJISTAR_100,CDLMORNINGSTAR_100,CDLONNECK_0,CDLPIERCING_100,CDLRICKSHAWMAN_100,CDLRISEFALL3METHODS_0,CDLRISEFALL3METHODS_100,CDLSEPARATINGLINES_0,CDLSEPARATINGLINES_100,CDLSHOOTINGSTAR_0,CDLSHORTLINE_0,CDLSHORTLINE_100,CDLSPINNINGTOP_0,CDLSPINNINGTOP_100,CDLSTALLEDPATTERN_0,CDLSTICKSANDWICH_100,CDLTAKURI_100,CDLTASUKIGAP_100,CDLTHRUSTING_0,CDLTRISTAR_0,CDLTRISTAR_100,CDLXSIDEGAP3METHODS_0,CDLXSIDEGAP3METHODS_100,volume_adi,volume_obv,volume_cmf,volume_fi,volume_em,volume_sma_em,volume_vpt,volume_vwap,volume_mfi,volume_nvi,volatility_bbm,volatility_bbh,volatility_bbl,volatility_bbw,volatility_bbp,volatility_bbhi,volatility_bbli,volatility_kcc,volatility_kch,volatility_kcl,volatility_kcw,volatility_kcp,volatility_kchi,volatility_kcli,volatility_dcl,volatility_dch,volatility_dcm,volatility_dcw,volatility_dcp,volatility_atr,volatility_ui,trend_macd,trend_macd_signal,trend_macd_diff,trend_sma_fast,trend_sma_slow,trend_ema_fast,trend_ema_slow,trend_vortex_ind_pos,trend_vortex_ind_neg,trend_vortex_ind_diff,trend_trix,trend_mass_index,trend_dpo,trend_kst,trend_kst_sig,trend_kst_diff,trend_ichimoku_conv,trend_ichimoku_base,trend_ichimoku_a,trend_ichimoku_b,trend_stc,trend_adx,trend_adx_pos,trend_adx_neg,trend_cci,trend_visual_ichimoku_a,trend_visual_ichimoku_b,trend_aroon_up,trend_aroon_down,trend_aroon_ind,trend_psar_up,trend_psar_down,trend_psar_up_indicator,trend_psar_down_indicator,momentum_rsi,momentum_stoch_rsi,momentum_stoch_rsi_k,momentum_stoch_rsi_d,momentum_tsi,momentum_uo,momentum_stoch,momentum_stoch_signal,momentum_wr,momentum_ao,momentum_roc,momentum_ppo,momentum_ppo_signal,momentum_ppo_hist,momentum_pvo,momentum_pvo_signal,momentum_pvo_hist,momentum_kama,others_dr,others_dlr,others_cr
t,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,

In [3]:
data = df.copy()

# Standardize Data

In [4]:
targets = np.array(data['Fdirection'].values.astype(int))
data_input = data.drop(['Fdirection'],axis=1)

In [5]:
scaler = StandardScaler()
scaler.fit(data_input)
scaled_inputs = scaler.transform(data_input)

# Prepare Windows

In [6]:
X , Y = None, None
X = []
Y = []

In [7]:
window = 15
for i in range (window,scaled_inputs.shape[0]):
    X.append(scaled_inputs[i-window:i])
    Y.append(targets[i-1])

# Split test-train

In [8]:
X , Y = np.array(X) , np.array(Y)

In [9]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size= 0.2,shuffle=True)

In [10]:
del X,Y

In [37]:
del x_train, x_test, y_train, y_test 

In [11]:
x_train.shape

(40687, 15, 283)

In [12]:
x_test.shape

(10172, 15, 283)

In [13]:
class Time2Vector(Layer):
  def __init__(self, seq_len, **kwargs):
    super(Time2Vector, self).__init__()
    self.seq_len = seq_len

  def build(self, input_shape):
    self.weights_linear = self.add_weight(name='weight_linear',
                                shape=(int(self.seq_len),),
                                initializer='uniform',
                                trainable=True)
    
    self.bias_linear = self.add_weight(name='bias_linear',
                                shape=(int(self.seq_len),),
                                initializer='uniform',
                                trainable=True)
    
    self.weights_periodic = self.add_weight(name='weight_periodic',
                                shape=(int(self.seq_len),),
                                initializer='uniform',
                                trainable=True)

    self.bias_periodic = self.add_weight(name='bias_periodic',
                                shape=(int(self.seq_len),),
                                initializer='uniform',
                                trainable=True)

  def call(self, x):
    #x = tf.math.reduce_mean(x[:,:,:285], axis=-1) # Convert (batch, seq_len, 5) to (batch, seq_len)
    x = tf.math.reduce_mean(x, axis=-1) # Convert (batch, seq_len, 5) to (batch, seq_len)
    time_linear = self.weights_linear * x + self.bias_linear
    time_linear = tf.expand_dims(time_linear, axis=-1) # (batch, seq_len, 1)
    
    time_periodic = tf.math.sin(tf.multiply(x, self.weights_periodic) + self.bias_periodic)
    time_periodic = tf.expand_dims(time_periodic, axis=-1) # (batch, seq_len, 1)
    return tf.concat([time_linear, time_periodic], axis=-1) # (batch, seq_len, 2)
  

In [14]:
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    # Normalization and Attention
    x = layers.LayerNormalization(epsilon=1e-6)(inputs)
    x = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout
    )(x, x)
    x = layers.Dropout(dropout)(x)
    res = x + inputs

    # Feed Forward Part
    x = layers.LayerNormalization(epsilon=1e-6)(res)
    x = layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(x)
    x = layers.Dropout(dropout)(x)
    x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
    return x + res

In [15]:
def build_model(
    input_shape,
    head_size,
    num_heads,
    ff_dim,
    num_transformer_blocks,
    mlp_units,
    dropout=0,
    mlp_dropout=0,
):
    #inputs = keras.Input(shape=input_shape)
    #x = inputs
    time_embedding = Time2Vector(window)
    in_seq = Input(shape=(window, x_train.shape[2]))
    x = time_embedding(in_seq)
    x = Concatenate(axis=-1)([in_seq, x])
    
    for _ in range(num_transformer_blocks):
        x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)

    x = layers.GlobalAveragePooling1D(data_format="channels_first")(x)
    for dim in mlp_units:
        x = layers.Dense(dim, activation="relu")(x)
        x = layers.Dropout(mlp_dropout)(x)
    outputs = layers.Dense(1, activation="sigmoid")(x)
    return keras.Model(in_seq, outputs)

## Train and evaluate

In [44]:
del model

In [27]:
x_train.shape[2]

283

In [16]:
input_shape = x_train.shape[1:]

model = build_model(
    input_shape,
    head_size=256,
    num_heads=4,
    ff_dim=4,
    num_transformer_blocks=4,
    mlp_units=[128],
    mlp_dropout=0.2,
    dropout=0.2,
)

model.compile(
    loss="binary_crossentropy",
    optimizer=keras.optimizers.Adam(learning_rate=1e-4),
    metrics=["accuracy"],
)
model.summary()

callbacks = [keras.callbacks.EarlyStopping(patience=20, restore_best_weights=True)]


#model.evaluate(x_test, y_test, verbose=1)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 15, 283)]    0           []                               
                                                                                                  
 time2_vector (Time2Vector)     (None, 15, 2)        60          ['input_1[0][0]']                
                                                                                                  
 concatenate (Concatenate)      (None, 15, 285)      0           ['input_1[0][0]',                
                                                                  'time2_vector[0][0]']           
                                                                                                  
 layer_normalization (LayerNorm  (None, 15, 285)     570         ['concatenate[0][0]']        

 eadAttention)                                                    'layer_normalization_6[0][0]']  
                                                                                                  
 dropout_6 (Dropout)            (None, 15, 285)      0           ['multi_head_attention_3[0][0]'] 
                                                                                                  
 tf.__operators__.add_6 (TFOpLa  (None, 15, 285)     0           ['dropout_6[0][0]',              
 mbda)                                                            'tf.__operators__.add_5[0][0]'] 
                                                                                                  
 layer_normalization_7 (LayerNo  (None, 15, 285)     570         ['tf.__operators__.add_6[0][0]'] 
 rmalization)                                                                                     
                                                                                                  
 conv1d_6 

In [17]:
model.fit(
    x_train,
    y_train,
    validation_split=0.2,
    epochs=10,
    batch_size=64,
    callbacks=callbacks,
    shuffle=True
)

Epoch 1/10
509/509 [==============================] - 123s 236ms/step - loss: 0.6935 - accuracy: 0.5015 - val_loss: 0.6929 - val_accuracy: 0.5135
Epoch 2/10
509/509 [==============================] - 125s 245ms/step - loss: 0.6934 - accuracy: 0.5015 - val_loss: 0.6931 - val_accuracy: 0.5039
Epoch 3/10
509/509 [==============================] - 125s 245ms/step - loss: 0.6929 - accuracy: 0.5096 - val_loss: 0.6927 - val_accuracy: 0.5143
Epoch 4/10
509/509 [==============================] - 127s 249ms/step - loss: 0.6926 - accuracy: 0.5145 - val_loss: 0.6923 - val_accuracy: 0.5238
Epoch 5/10
509/509 [==============================] - 125s 246ms/step - loss: 0.6920 - accuracy: 0.5202 - val_loss: 0.6923 - val_accuracy: 0.5144
Epoch 6/10
509/509 [==============================] - 125s 246ms/step - loss: 0.6907 - accuracy: 0.5288 - val_loss: 0.6927 - val_accuracy: 0.5157
Epoch 7/10
509/509 [==============================] - 127s 249ms/step - loss: 0.6883 - accuracy: 0.5405 - val_loss: 0.6953 -

https://keras.io/examples/timeseries/timeseries_transformer_classification/

https://towardsdatascience.com/stock-predictions-with-state-of-the-art-transformer-and-time-embeddings-3a4485237de6

https://towardsdatascience.com/the-time-series-transformer-2a521a0efad3